In [1]:
!pip install requests
!pip install beautifulsoup4
!pip install selenium


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [16]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

url = 'https://camp-fire.jp/projects?project_status%5B%5D=closed&sort=popular'
response = requests.get(url)

response.encoding = 'utf-8'

# ウェブサイトのHTMLコンテンツを取得
html_content = response.text

print(html_content)

# BeautifulSoupオブジェクトを作成
soup = BeautifulSoup(html_content, 'html.parser')

# 特定のクラスを持つdivタグを検索
target_divs = soup.find_all('div', class_='value svelte-1meccjp')

# 取得したタグの内容を表示
for div in target_divs:
    print(div.text)


chrome open
chrome close


In [5]:
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Chromeのオプションを設定
options = Options()
options.add_argument('--headless')  # ヘッドレスモードを有効化
options.add_argument('--disable-gpu')  # GPUハードウェアアクセラレーションを無効化（必要に応じて）
options.add_argument('--window-size=1920x1080')  # ウィンドウサイズを指定（必要に応じて）

# WebDriverのパスとオプションを設定
driver = webdriver.Chrome(options=options)

# ページを指定
# TODO ページ範囲を指定して回すように変更
PAGE = 1

paginate_url = 'https://camp-fire.jp/projects?page=' + str(PAGE) + '&project_status%5B%5D=closed'

# 指定URLを開く（ヘッドレスモードなのでウィンドウは開かない）
driver.get(paginate_url)
print('page_url: ' + driver.current_url) 

# 実行後のHTMLを取得
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# タグ検索
container = soup.find('div', class_='container')

print(container)

if container:
    boxes4 = container.find('div', class_='boxes4')
    if boxes4:
        boxes = boxes4.find_all('div', class_='box')
        for box in boxes:
            if box:
                box_thumbnail = box.find('div', class_='box-thumbnail')
                if box_thumbnail:
                    script_tag = box_thumbnail.find('script')
                    if script_tag:
                        # scriptタグのテキストから数字を抽出
                        number = re.findall(r'/projects/(\d+)', script_tag.string)[0]
                    else:
                        a_tags = box_thumbnail.find_all('a', href=True)
                        for a_tag in a_tags:
                            # aタグのhref属性からプロジェクトID（数字）を抽出
                            href = a_tag['href']
                            match = re.search(r'/projects/view/(\d+)', href)
                            if match:
                                number = match.group(1)
                                
                    url = 'https://camp-fire.jp/projects/' + number
                    print('URL: ', url)  # 最初にマッチした数字を出力

                    driver_detail = webdriver.Chrome(options=options)

                    driver_detail.get(url)
                    html_detail = driver_detail.page_source
                    soup_detail = BeautifulSoup(html_detail, 'html.parser')

                    # タイトル
                    labels = soup_detail.find_all('label', class_='project-name')
                    if labels:
                        print('タイトル: ',labels[0].text)
                    else:
                        print('タイトル: ')

                    # 支援総額
                    target_divs = soup_detail.find_all('div', class_='another-default')
                    
                    for div in target_divs:
                        total = div.find('strong')
                        print('支援総額: ',total.text)
                    
                    # 目標金額
                    target_divs = soup_detail.find_all('div', class_='target tb-none sp-none')
                    
                    for div in target_divs:
                        target = div.find('span')
                        print('目標金額: ',target.text.replace('JPY', ''))

                    # 支援人数
                    target_divs = soup_detail.find_all('div', class_='patron inner')
                    for div in target_divs:
                        number_supporters = div.find('strong')
                        print('支援人数: ',number_supporters.text)

                    # ステータス
                    target_divs = soup_detail.find_all('div', class_='rest inner')
                    for div in target_divs:
                        status = div.find('strong')
                        print('ステータス: ',status.text)
                    
                    driver_detail.quit() 
                else:
                    print("box-thumbnailクラスが見つかりません")
            else:
                print("box-in project-card-contentクラスが見つかりません")
    else:
        print("boxes4クラスが見つかりません")
else:
    print("containerクラスが見つかりません")
driver.quit()

page_url: https://camp-fire.jp/projects/search?project_status=closed&page=1
<div class="container svelte-gl90xg"><section class="svelte-gl90xg"><h2 class="title svelte-kvmbqf">人気のプロジェクト</h2> <ol class="card-area svelte-gl90xg"><li class="card-wrapper"><a class="card svelte-1ljob97" data-gtm-data-brand="customer_success" data-gtm-data-category="category_fashion" data-gtm-data-dimension8="project" data-gtm-data-id="720802" data-gtm-data-list="projects_popular" data-gtm-data-name="Rain帽商品化" data-gtm-data-position="1" data-gtm-vis-first-on-screen9589075_766="652" data-gtm-vis-has-fired9589075_766="1" data-gtm-vis-recent-on-screen9589075_766="652" data-gtm-vis-total-visible-time9589075_766="100" data-href="/projects/view/720802?list=projects_popular" data-title="Rain帽商品化" data_project_id="720802" tabindex="0" title="Rain帽商品化"><div class="image svelte-1ljob97"><div class="img-container svelte-oij4dc"><img alt="" class="card-img svelte-oij4dc" loading="lazy" src="https://static.camp-fire.jp/u

In [22]:
# FIXME ページをまたぐスクレイピングができない
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# Constants
BASE_URL = 'https://camp-fire.jp/projects'
MAX_PAGE = 10

def init_webdriver():
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--window-size=1920x1080')
    return webdriver.Chrome(options=options)

def extract_number_from_script(script_text):
    return re.findall(r'/projects/(\d+)', script_text)[0] if script_text else None

def extract_number_from_href(href):
    match = re.search(r'/projects/view/(\d+)', href)
    return match.group(1) if match else None

def print_project_details(soup_detail):
    labels = soup_detail.find_all('label', class_='project-name')
    print('タイトル: ', labels[0].text if labels else 'N/A')

    # ノーオペレーション ラムダ関数を定義
    noop = lambda x: x

    divs_to_extract = [
        ('strong', 'another-default', '支援総額', noop),
        ('span', 'target tb-none sp-none', '目標金額', lambda x: x.replace('JPY', '')),
        ('strong', 'patron inner', '支援人数', noop),
        ('strong', 'rest inner', 'ステータス', noop)
    ]

    for tag, class_name, label, transform_func in divs_to_extract:
        divs = soup_detail.find_all('div', class_=class_name)
        for div in divs:
            # 目標金額のときのみspanあとは
            value = div.find(tag)
            text = value.text if value else None
            
            print(f'{label}: ', transform_func(value.text) if value else 'N/A')

def process_page(driver, page_number):
    url = f'{BASE_URL}?page={page_number}&project_status%5B%5D=closed'
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    container = soup.find('div', class_='container')
    if not container:
        print("containerクラスが見つかりません")
        return

    boxes = container.select('.boxes4 .box')
    for box in boxes:
        box_thumbnail = box.find('div', class_='box-thumbnail')
        if not box_thumbnail:
            print("box-thumbnailクラスが見つかりません")
            continue

        script_tag = box_thumbnail.find('script')
        number = extract_number_from_script(script_tag.string) if script_tag else None

        if not number:
            for a_tag in box_thumbnail.find_all('a', href=True):
                number = extract_number_from_href(a_tag['href'])
                if number:
                    break

        if number:
            detail_driver = init_webdriver()
            print('url: ' + f'{BASE_URL}/{number}')
            detail_driver.get(f'{BASE_URL}/{number}')
            print_project_details(BeautifulSoup(detail_driver.page_source, 'html.parser'))
            detail_driver.quit()
        else:
            print("Project number not found")

def main():
    driver = init_webdriver()
    for i in range(MAX_PAGE):
        process_page(driver, i)
    driver.quit()

if __name__ == '__main__':
    main()


<div class="container svelte-gl90xg"><section class="svelte-gl90xg"><h2 class="title svelte-kvmbqf">人気のプロジェクト</h2> <ol class="card-area svelte-gl90xg"><li class="card-wrapper"><a class="card svelte-1ljob97" data-gtm-data-brand="customer_success" data-gtm-data-category="category_fashion" data-gtm-data-dimension8="project" data-gtm-data-id="720802" data-gtm-data-list="projects_popular" data-gtm-data-name="Rain帽商品化" data-gtm-data-position="1" data-gtm-vis-first-on-screen9589075_766="1108" data-gtm-vis-has-fired9589075_766="1" data-gtm-vis-recent-on-screen9589075_766="1108" data-gtm-vis-total-visible-time9589075_766="100" data-href="/projects/view/720802?list=projects_popular" data-title="Rain帽商品化" data_project_id="720802" tabindex="0" title="Rain帽商品化"><div class="image svelte-1ljob97"><div class="img-container svelte-oij4dc"><img alt="" class="card-img svelte-oij4dc" loading="lazy" src="https://static.camp-fire.jp/uploads/project_version/image/1178904/2c78494c-06ab-4939-9fac-969615f9e4fd.

KeyboardInterrupt: 

In [37]:
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# Constants
BASE_URL = 'https://camp-fire.jp/projects'
MAX_PAGE = 1

def init_webdriver():
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--window-size=1920x1080')
    return webdriver.Chrome(options=options)

def extract_number_from_href(href):
    match = re.search(r'/projects/view/(\d+)', href)
    return match.group(1) if match else None

def print_project_details(soup_detail):
    divs_to_extract = [
        ('strong', 'another-default', '支援総額', lambda x: x),
        ('span', 'target tb-none sp-none', '目標金額', lambda x: x.replace('JPY', '')),
        ('strong', 'patron inner', '支援人数', lambda x: x),
        ('strong', 'rest inner', 'ステータス', lambda x: x)
    ]

    for tag, class_name, label, transform_func in divs_to_extract:
        divs = soup_detail.find_all('div', class_=class_name)
        for div in divs:
            value = div.find(tag)
            print(f'{label}: ', transform_func(value.text) if value else 'N/A')

def process_page(driver, page_number):
    url = f'{BASE_URL}?page={page_number}&project_status%5B%5D=closed'
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    container = soup.find('div', class_='container')
    if not container:
        print("containerクラスが見つかりません")
        return

    cards = container.find_all('li', class_='card-wrapper')
    for card in cards:
        project_href = card.find('a', class_='card')['data-href']
        project_id = extract_number_from_href(project_href)
        if project_id:
            project_url = f'https://camp-fire.jp{project_href}'
            driver.get(project_url)
            project_soup = BeautifulSoup(driver.page_source, 'html.parser')
            print('タイトル: ', card.find('a', class_='card')['data-title'])
            print_project_details(project_soup)

def main():
    driver = init_webdriver()
    for i in range(1, MAX_PAGE + 1):
        process_page(driver, i)
    driver.quit()

if __name__ == '__main__':
    main()


タイトル:  Rain帽商品化
支援総額:  3,000
目標金額:  500,000
支援人数:  1
ステータス:  終了
タイトル:  持ち運べる発電所！多機能ソーラーバッテリー「FREEK3」
支援総額:  87,508
目標金額:  50,000
支援人数:  9
ステータス:  終了
タイトル:  独立を目指す造園家のための機材購入プロジェクト
支援総額:  0
目標金額:  100,000
支援人数:  0
ステータス:  終了
タイトル:  プエラの絶対値【渚一夏】生誕祭を盛り上げたい！
支援総額:  1,006,000
目標金額:  500,000
支援人数:  67
ステータス:  終了
タイトル:  ワインバルトパの開店祝いで植栽をプレゼントしよう！
支援総額:  815,000
目標金額:  300,000
支援人数:  54
ステータス:  終了
タイトル:  地域密着で看護師が必ずいるデイサービス
支援総額:  0
目標金額:  5,000,000
支援人数:  0
ステータス:  終了
タイトル:  「女性リーダーの為の完全会員制BAR(南青山)」初期会員・限定募集
支援総額:  2,978,611
目標金額:  500,000
支援人数:  122
ステータス:  終了
タイトル:  花華生誕祭開催プロジェクト
支援総額:  560,000
目標金額:  350,000
支援人数:  29
ステータス:  終了
タイトル:  テキスタイルエンジニアが作る「ちょっと変わった」カットソー
支援総額:  288,800
目標金額:  200,000
支援人数:  10
ステータス:  終了
タイトル:  家事代行スタート！令和6年、京都・奈良エリアでサービス開始プロジェクト
支援総額:  0
目標金額:  150,000
支援人数:  0
ステータス:  終了
タイトル:  音楽で御代田町を盛り上げたい！ 
支援総額:  500
目標金額:  100,000
支援人数:  1
ステータス:  終了
タイトル:  安心安全な夜職(性風俗は除く)の人材紹介
支援総額:  0
目標金額:  5,000,000
支援人数:  0
ステータス:  終了
タイトル:  無化学肥料・無農薬で育てた安心で美味しい野菜たちを、沢山の方々に提供し続けたい
支援

In [45]:
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# Constants
BASE_URL = 'https://camp-fire.jp/projects'

# 1ページあたり20個(20×MAX_PAGE)
MAX_PAGE = 50


def init_webdriver():
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--window-size=1920x1080')
    return webdriver.Chrome(options=options)

def extract_number_from_href(href):
    match = re.search(r'/projects/view/(\d+)', href)
    return match.group(1) if match else None

def get_project_details(soup_detail):
    divs_to_extract = [
        ('strong', 'another-default', '支援総額', lambda x: x.replace(',', '')),
        ('span', 'target tb-none sp-none', '目標金額', lambda x: x.replace('JPY', '').replace(',', '')),
        ('strong', 'patron inner', '支援人数', lambda x: x),
        ('strong', 'rest inner', 'ステータス', lambda x: x)
    ]

    details = []

    for tag, class_name, label, transform_func in divs_to_extract:
        divs = soup_detail.find_all('div', class_=class_name)
        for div in divs:
            value = div.find(tag)
            details.append(transform_func(value.text) if value else 'N/A')

    return details

def process_page(driver, page_number):
    url = f'{BASE_URL}?page={page_number}&project_status%5B%5D=closed'
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    container = soup.find('div', class_='container')
    if not container:
        print("containerクラスが見つかりません")
        return

    cards = container.find_all('li', class_='card-wrapper')
    for card in cards:
        project_href = card.find('a', class_='card')['data-href']
        project_id = extract_number_from_href(project_href)
        if project_id:
            project_url = f'https://camp-fire.jp{project_href}'
            driver.get(project_url)
            project_soup = BeautifulSoup(driver.page_source, 'html.parser')
            title = card.find('a', class_='card')['data-title']
            details = get_project_details(project_soup)
            print(','.join([title] + details))

def main():
    driver = init_webdriver()
    for i in range(1, MAX_PAGE + 1):
        process_page(driver, i)
    driver.quit()

if __name__ == '__main__':
    main()


Rain帽商品化,3000,500000,1,終了
持ち運べる発電所！多機能ソーラーバッテリー「FREEK3」,87508,50000,9,終了
独立を目指す造園家のための機材購入プロジェクト,0,100000,0,終了
プエラの絶対値【渚一夏】生誕祭を盛り上げたい！,1006000,500000,67,終了
ワインバルトパの開店祝いで植栽をプレゼントしよう！,815000,300000,54,終了
地域密着で看護師が必ずいるデイサービス,0,5000000,0,終了
「女性リーダーの為の完全会員制BAR(南青山)」初期会員・限定募集,2978611,500000,122,終了
花華生誕祭開催プロジェクト,560000,350000,29,終了
テキスタイルエンジニアが作る「ちょっと変わった」カットソー,288800,200000,10,終了
家事代行スタート！令和6年、京都・奈良エリアでサービス開始プロジェクト,0,150000,0,終了
音楽で御代田町を盛り上げたい！ ,500,100000,1,終了
安心安全な夜職(性風俗は除く)の人材紹介,0,5000000,0,終了
無化学肥料・無農薬で育てた安心で美味しい野菜たちを、沢山の方々に提供し続けたい,1390500,8503000,49,終了
「ゆるいキーウィ」ぬいぐるみ化プロジェクト第3弾！,6790200,2200000,616,終了
欲しかった中華鍋。大容量でも理想の温度を実現｜深型鉄フライパン28cm,155600,10000,10,終了
第1弾MOS GOES TO YOUR TOWN<あなたの街にMOSが行きます>,979258,2000000,121,終了
【これがゴミ箱の最終形態】ゴミ箱のあるあるをマルっとカバー！スマートゴミ箱登場,960500,100000,57,終了
フェイクニュースや報道しない自由に抗う、新しいメディアを作りたい！,5000,300000,1,終了
もっとたくさんの野良猫を救いたい！,6000,300000,2,終了
高校生が地域貢献のために企画する「ゴミ拾い×謎解き」イベントに力を貸してください,24500,50000,7,終了
【甲南大学男子ラクロス部】韓国遠征で挑む！1部昇格への貴重な機会,1048000,1000000,56,終了
岡山県津山市で無農薬野菜を栽培し